# ML Modeling: Epoch 2
> - States with highest exclusion ratios
> - Specialty

In [9]:
import pandas as pd
import numpy as np

In [15]:
# load id_df from efs
id_df = pd.read_csv('~/SageMaker/efs/DrFraud/data/id_df_top_states.csv', index_col=0)
id_df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,...,total_day_supply,total_drug_cost,bene_count_ge65,bene_count_ge65_suppress_flag,total_claim_count_ge65,ge65_suppress_flag,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,year
0,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,ISOSORBIDE MONONITRATE ER,ISOSORBIDE MONONITRATE,NaN,...,307,171.59,NaN,*,NaN,*,NaN,NaN,NaN,2013
1,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,LEVOFLOXACIN,LEVOFLOXACIN,26.0,...,165,227.10,15.0,NaN,15.0,NaN,15.0,106.0,159.72,2013
2,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,LISINOPRIL,LISINOPRIL,17.0,...,570,100.37,NaN,#,NaN,#,NaN,NaN,NaN,2013
3,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,METOPROLOL TARTRATE,METOPROLOL TARTRATE,28.0,...,916,154.65,NaN,#,NaN,#,NaN,NaN,NaN,2013
4,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,PREDNISONE,PREDNISONE,14.0,...,133,44.72,NaN,*,NaN,*,NaN,NaN,NaN,2013


In [16]:
# load LEIE from efs
LEIE_df = pd.read_csv('~/SageMaker/efs/DrFraud/data/LEIE_df_top_states.csv', index_col=0)
LEIE_df.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,...,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,year,month,key_present
0,NaN,NaN,,ACCELERATED BEHAVIOR MANAGEMEN,OTHER BUSINESS,COMM MNTL HLTH CNTR,NaN,1902198435,NaN,6148 W SAHARA AVENUE,...,NV,89146,1128a1,20160120,0,0,NaN,2016,1,True
1,NaN,NaN,,"ADVANCED COUNSELING GROUP, LLC",OTHER BUSINESS,COUNSELING CENTER,NaN,1164669479,NaN,"3170 E SUNSET RD, SUITE A",...,NV,89120,1128a1,20161020,0,0,NaN,2016,10,True
2,NaN,NaN,,ANCHORAGE ADULT DAY SERVICES,OTHER BUSINESS,HOME HEALTH AGENCY,NaN,1649524182,NaN,"360 BONIFACE PKWY, UNIT B8",...,AK,99504,1128a1,20160420,0,0,NaN,2016,4,True
3,NaN,NaN,,ARMS OF GRACE HUMANITARIAN SER,OTHER BUSINESS,COMMUNITY HLTH CTR (,NaN,1275856452,NaN,"2821 CRENSHAW BLVD, 100",...,CA,90016,1128a1,20180118,0,0,NaN,2018,1,True
4,NaN,NaN,,"BECAUSE WE CARE, LLC",OTHER BUSINESS,COMM MNTL HLTH CNTR,NaN,1437449139,NaN,6748 LAVENDER LILLY LANE,...,NV,89084,1128a1,20180719,0,0,NaN,2018,7,True


In [17]:
df = id_df[['year',
            'npi',
            'specialty_description',
            'nppes_provider_state',
            'nppes_provider_city',]]
df.head()

,year,npi,specialty_description,nppes_provider_state,nppes_provider_city
0,2013,1003000126,Internal Medicine,MD,CUMBERLAND
1,2013,1003000126,Internal Medicine,MD,CUMBERLAND
2,2013,1003000126,Internal Medicine,MD,CUMBERLAND
3,2013,1003000126,Internal Medicine,MD,CUMBERLAND
4,2013,1003000126,Internal Medicine,MD,CUMBERLAND


In [18]:
df1 = df.drop_duplicates().reset_index(drop=True)
df1.head()

,year,npi,specialty_description,nppes_provider_state,nppes_provider_city
0,2013,1003000126,Internal Medicine,MD,CUMBERLAND
1,2013,1003000167,Dentist,NV,DAYTON
2,2013,1003000639,Cardiac Surgery,CA,LOS ANGELES
3,2013,1003001017,Dermatology,CA,LA PALMA
4,2013,1003001363,Anesthesiology,CA,EL CENTRO


In [19]:
df1.shape

(125915, 5)

In [20]:
year_excluded = LEIE_df[['year','NPI']].drop_duplicates()
year_excluded.head()

,year,NPI
0,2016,1902198435
1,2016,1164669479
2,2016,1649524182
3,2018,1275856452
4,2018,1437449139


In [22]:
print('Shape of `year_excluded_df`: ', year_excluded.shape)

Shape of `year_excluded_df`:  (729, 2)


In [24]:
a = set(df1['npi'].values)
b = set(year_excluded['NPI'].values)
print('unique npis in Medicare Part D: ', len(a))
print('unique npis in LEIE: ', len(b))
print('npis common to both Medicare Part D and LEIE: ',len(a.intersection(b)))

unique npis in Medicare Part D:  125824
unique npis in LEIE:  722
npis common to both Medicare Part D and LEIE:  190


In [26]:
# Label targets
exclusion_years = sorted(list(set(year_excluded['year'])))
for year in exclusion_years:
    excluded_npis_that_year = set(year_excluded['NPI'][year_excluded['year'] == year])
    df1['exlcluded'] = np.where((df1['year'] < year) & (df1['npi'].isin(excluded_npis_that_year)), True, df1['exlcluded'])
df1.head()

KeyError: 'exlcluded'

In [108]:
# save df to efs
def save_to_efs(df, f_name):
    efs_path = '~/SageMaker/efs/DrFraud/data/{}'.format(f_name)
    df.to_csv(efs_path, sep=',')

save_to_efs(df1, 'specialty_state_labeled.csv')

In [76]:
print('number of npis labeled as `excluded` from the Medicare Part D: ', df1[df1['exlcluded'] == True].shape[0])

number of npis labeled as `excluded` from the Medicare Part D:  1025


** Great, looks like the labeling works!!! The number may not match exactly because some doctors may be included to particiapte in the Medicare Part D.**

## ML Model - Specialty
---

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [86]:
# build feature matrix
specialty_df = pd.get_dummies(df1[['year', 'npi', 'specialty_description','nppes_provider_state']],\
                                  columns=['specialty_description','nppes_provider_state']).\
                                  set_index(['year','npi'], drop=True)
specialty_df.head()

specialty_description_Acupuncturist  \
year npi                                               
2013 1003000126                                    0   
     1003000142                                    0   
     1003000167                                    0   
     1003000282                                    0   
     1003000407                                    0   

                 specialty_description_Addiction Medicine  \
year npi                                                    
2013 1003000126                                         0   
     1003000142                                         0   
     1003000167                                         0   
     1003000282                                         0   
     1003000407                                         0   

                 specialty_description_Adult Companion  \
year npi                                                 
2013 1003000126                                      0   
     1003000142                                      0   
     1003000167                                      0   
     1003000282                                      0   
     1003000407                                      0   

                 specialty_description_All Other Suppliers  \
year npi                                                     
2013 1003000126                                          0   
     1003000142                                          0   
     1003000167                                          0   
     1003000282                                          0   
     1003000407                                          0   

                 specialty_description_Allergy/Immunology  \
year npi                                                    
2013 1003000126                                         0   
     1003000142                                         0   
     1003000167                                         0   
     1003000282                                         0   
     1003000407                                         0   

                 specialty_description_Ambulance Service Supplier  \
year npi                                                            
2013 1003000126                                                 0   
     1003000142                                                 0   
     1003000167                                                 0   
     1003000282                                                 0   
     1003000407                                                 0   

                 specialty_description_Ambulatory Surgical Center  \
year npi                                                            
2013 1003000126                                                 0   
     1003000142                                                 0   
     1003000167                                                 0   
     1003000282                                                 0   
     1003000407                                                 0   

                 specialty_description_Anesthesiologist Assistants  \
year npi                                                             
2013 1003000126                                                  0   
     1003000142                                                  0   
     1003000167                                                  0   
     1003000282                                                  0   
     1003000407                                                  0   

                 specialty_description_Anesthesiology  \
year npi                                                
2013 1003000126                                     0   
     1003000142                                     1   
     1003000167                                     0   
     1003000282                                     0   
     1003000407                                     0   

                 specialty_description_Assisted Living Facility  \


In [89]:
X = specialty_df.values
y = df1['exlcluded'].values
print("X shape: ", X.shape)

X shape:  (808603, 263)


In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=123)
rf_classifier = RandomForestClassifier(n_estimators = 100, n_jobs=-1)

In [155]:
# out of the box
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

# Assess Perforamnce
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
my_confusion_matrix = np.array([[tp, fp],
                                [tn, fn]])
my_confusion_matrix

array([[     0,      2],
       [161514,    205]])

**_The results are bad! Which was expected given such an imbalance of classes._**  <br>
> Just how bad? Well... the worst case scenario would be to miss all `excluded` doctors proportionally to the train test split. In other words, 20% of the 1025, **`206`**. That's about how much we missed (look at `false negatives`). Keep in mind, the number will not match exactly due to random sampling during the train test split.

**Strategies to improve performance due to imbalanced classes problem:**
- Tweak sample weights and rerun again
- Focus on states and even specialties that have the lowest issue of imblanaced classes.
- Use fuzzywuzzy to match up by name, specialty, and city.

### Tweak the sample weights and try to run the model again
---

In [156]:
# Adjust sample weights
factor = 20
sample_weight = np.array([factor if label == 1 else 1 for label in y_train])
rf_classifier.fit(X_train, y_train, sample_weight=sample_weight)
y_pred = rf_classifier.predict(X_test)

# Assess Perforamnce
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
my_confusion_matrix = np.array([[tp, fp],
                                [tn, fn]])
my_confusion_matrix

array([[     0,     23],
       [161493,    205]])

**_Sample weights doesn't impact false negative rate_**